In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('classic')

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [3]:
import os
import cv2
from PIL import Image
import numpy as np

In [4]:
image_directory = '299aug-nor-macular/'
SIZE = 128
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

In [5]:
macular_degeneration_images = os.listdir(image_directory + 'macular-degeneration/')
for i, image_name in enumerate(macular_degeneration_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'macular-degeneration/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

In [6]:
normal_images = os.listdir(image_directory + 'normal/')
for i, image_name in enumerate(normal_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

In [7]:
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [9]:
#Without scaling (normalize) the training may not converge. 
#Normalization is a rescaling of the data from the original range 
#so that all values are within the range of 0 and 1.
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

#Do not do one-hot encoding as it generates a shape of (num, 2)
#But the network expects an input of (num, 1) for the last layer for binary classification
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [10]:
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(1))
model.add(Activation('sigmoid'))  
#Do not use softmax for binary classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins. 

In [11]:
#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#But, also good for binary mutually exclusive (cat or not cat). 

model.compile(loss='binary_crossentropy',
              optimizer='adam',             #also try adam
              metrics=['accuracy'])


In [12]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0

In [13]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
400/400 [==============================] - 76s 188ms/step - loss: 0.4986 - accuracy: 0.7456 - val_loss: 0.4539 - val_accuracy: 0.7944
Epoch 2/10
400/400 [==============================] - 76s 191ms/step - loss: 0.4008 - accuracy: 0.8227 - val_loss: 0.5351 - val_accuracy: 0.7219
Epoch 3/10
400/400 [==============================] - 75s 188ms/step - loss: 0.3837 - accuracy: 0.8314 - val_loss: 0.4666 - val_accuracy: 0.7600
Epoch 4/10
400/400 [==============================] - 77s 193ms/step - loss: 0.3640 - accuracy: 0.8425 - val_loss: 0.4500 - val_accuracy: 0.7681
Epoch 5/10
400/400 [==============================] - 84s 209ms/step - loss: 0.3488 - accuracy: 0.8491 - val_loss: 0.4583 - val_accuracy: 0.7569
Epoch 6/10
400/400 [==============================] - 81s 203ms/step - loss: 0.3302 - accuracy: 0.8594 - val_loss: 0.3559 - val_accuracy: 0.8350
Epoch 7/10
400/400 [==============================] - 85s 212ms/step - loss: 0.2990 - accuracy: 0.8717 - val_loss: 0.3061 - val_ac

In [14]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
400/400 [==============================] - 79s 197ms/step - loss: 0.2486 - accuracy: 0.8895 - val_loss: 0.2964 - val_accuracy: 0.8637
Epoch 2/10
400/400 [==============================] - 79s 198ms/step - loss: 0.2259 - accuracy: 0.9052 - val_loss: 0.2690 - val_accuracy: 0.8788
Epoch 3/10
400/400 [==============================] - 79s 197ms/step - loss: 0.2219 - accuracy: 0.9047 - val_loss: 0.2935 - val_accuracy: 0.8737
Epoch 4/10
400/400 [==============================] - 77s 192ms/step - loss: 0.2168 - accuracy: 0.9038 - val_loss: 0.2670 - val_accuracy: 0.8794
Epoch 5/10
400/400 [==============================] - 76s 191ms/step - loss: 0.2052 - accuracy: 0.9094 - val_loss: 0.2406 - val_accuracy: 0.8894
Epoch 6/10
400/400 [==============================] - 84s 209ms/step - loss: 0.1981 - accuracy: 0.9158 - val_loss: 0.2480 - val_accuracy: 0.8869
Epoch 7/10
400/400 [==============================] - 80s 199ms/step - loss: 0.1834 - accuracy: 0.9217 - val_loss: 0.2445 - val_ac

In [15]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 5,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/5
400/400 [==============================] - 82s 204ms/step - loss: 0.1650 - accuracy: 0.9322 - val_loss: 0.2375 - val_accuracy: 0.8850
Epoch 2/5
400/400 [==============================] - 84s 209ms/step - loss: 0.1513 - accuracy: 0.9370 - val_loss: 0.3069 - val_accuracy: 0.8531
Epoch 3/5
400/400 [==============================] - 82s 205ms/step - loss: 0.1701 - accuracy: 0.9269 - val_loss: 0.2654 - val_accuracy: 0.8731
Epoch 4/5
400/400 [==============================] - 96s 240ms/step - loss: 0.1597 - accuracy: 0.9312 - val_loss: 0.2291 - val_accuracy: 0.8875
Epoch 5/5
400/400 [==============================] - 106s 266ms/step - loss: 0.1438 - accuracy: 0.9392 - val_loss: 0.2473 - val_accuracy: 0.8831


In [16]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 32, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
200/200 [==============================] - 107s 532ms/step - loss: 0.1329 - accuracy: 0.9450 - val_loss: 0.2237 - val_accuracy: 0.8988
Epoch 2/10
200/200 [==============================] - 106s 531ms/step - loss: 0.1334 - accuracy: 0.9444 - val_loss: 0.2219 - val_accuracy: 0.9000
Epoch 3/10
200/200 [==============================] - 112s 559ms/step - loss: 0.1187 - accuracy: 0.9533 - val_loss: 0.2267 - val_accuracy: 0.8956
Epoch 4/10
200/200 [==============================] - 106s 530ms/step - loss: 0.1102 - accuracy: 0.9563 - val_loss: 0.2295 - val_accuracy: 0.9031
Epoch 5/10
200/200 [==============================] - 110s 549ms/step - loss: 0.1190 - accuracy: 0.9536 - val_loss: 0.2363 - val_accuracy: 0.8900
Epoch 6/10
200/200 [==============================] - 108s 542ms/step - loss: 0.1097 - accuracy: 0.9547 - val_loss: 0.2210 - val_accuracy: 0.9050
Epoch 7/10
200/200 [==============================] - 107s 534ms/step - loss: 0.1106 - accuracy: 0.9548 - val_loss: 0.2122 -

In [17]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 32, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
200/200 [==============================] - 107s 534ms/step - loss: 0.1029 - accuracy: 0.9623 - val_loss: 0.2451 - val_accuracy: 0.8931
Epoch 2/10
200/200 [==============================] - 108s 540ms/step - loss: 0.0991 - accuracy: 0.9602 - val_loss: 0.2164 - val_accuracy: 0.9044
Epoch 3/10
200/200 [==============================] - 111s 553ms/step - loss: 0.0943 - accuracy: 0.9645 - val_loss: 0.2417 - val_accuracy: 0.8925
Epoch 4/10
200/200 [==============================] - 117s 584ms/step - loss: 0.0841 - accuracy: 0.9677 - val_loss: 0.4102 - val_accuracy: 0.8394
Epoch 5/10
200/200 [==============================] - 112s 560ms/step - loss: 0.0924 - accuracy: 0.9642 - val_loss: 0.2643 - val_accuracy: 0.8831
Epoch 6/10
200/200 [==============================] - 107s 537ms/step - loss: 0.0855 - accuracy: 0.9656 - val_loss: 0.2334 - val_accuracy: 0.9031
Epoch 7/10
200/200 [==============================] - 97s 487ms/step - loss: 0.0897 - accuracy: 0.9659 - val_loss: 0.2330 - 